In [40]:
import pandas as pd

def parse_price(s: str) -> float:
    assert s.endswith(" €")
    return float(s[:-2].replace(',', '.'))

DATA = pd.read_csv('Bakery sales.csv')
DATA['Quantity'] = pd.to_numeric(DATA['Quantity'], downcast='integer')
DATA['unit_price'] = DATA['unit_price'].map(parse_price)
DATA['total_price'] = DATA['Quantity'] * DATA['unit_price']
DATA.head()

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price,total_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1,0.90,0.90
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3,1.20,3.60
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2,1.20,2.40
3,5,2021-01-02,09:14,150041.0,PAIN,1,1.15,1.15
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5,1.20,6.00


In [39]:
TRANSACTIONS = DATA.groupby('ticket_number').agg({'article': list, 'total_price': 'sum', 'time': 'first'})
TRANSACTIONS.head()

,article,total_price,time
ticket_number,,,
150040.0,"[BAGUETTE, PAIN AU CHOCOLAT]",4.50,08:38
150041.0,"[PAIN AU CHOCOLAT, PAIN]",3.55,09:14
150042.0,[TRADITIONAL BAGUETTE],6.00,09:25
150043.0,"[BAGUETTE, CROISSANT]",5.10,09:25
150044.0,[BANETTE],1.05,09:27
